In [2]:
from serpapi import GoogleSearch
from pprint import pprint
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
import json
import os

In [63]:
#########################################################
# WARNING!                                              #
# Don't run with this API key unless 100% required      #
#########################################################

# API Key Store
# Travis: 7b49b198668e629ad2ea004850f238d18e56e6e6f3ed19a4d71430a670f4ba29
# Leon: a50a83b24d1e51c42d6567fb1bd517dadd4d4348957fe3450928060bbb400ada
# Konni: 

params = {
    "engine": "google_jobs",
    "q": "data scientist",
    "hl": "en",
    "api_key": "a50a83b24d1e51c42d6567fb1bd517dadd4d4348957fe3450928060bbb400ada",
    "gl": "uk",
    "location": "United States"
}

search = GoogleSearch(params)
results = search.get_dict()
jobs_results = results["jobs_results"]

https://serpapi.com/search


In [91]:
# Pretty Print the results
pprint(results)

{'chips': [{'options': [{'text': 'All'},
                        {'text': 'Data scientist', 'value': 'data scientist'},
                        {'text': 'Data science', 'value': 'data science'},
                        {'text': 'Manager', 'value': 'manager'},
                        {'text': 'Administrator', 'value': 'administrator'},
                        {'text': 'Analyst', 'value': 'analyst'},
                        {'text': 'Applied scientist',
                         'value': 'applied scientist'},
                        {'text': 'Biostatistician', 'value': 'biostatistician'},
                        {'text': 'Computer', 'value': 'computer'},
                        {'text': 'Consultant', 'value': 'consultant'},
                        {'text': 'Data analyst', 'value': 'data analyst'},
                        {'text': 'Data engineer', 'value': 'data engineer'},
                        {'text': 'Director', 'value': 'director'},
                        {'text': 'Engineer', 'valu

In [86]:
def merge_highlights(highlights: list) -> list:
    """
    A function that takes a list of dictionaries and strings and merges
    them into a list of individual strings by extracting the values with
    'item' tags and ignoring 'title'
    """
    output = []
    for highlight in highlights:
        
        if type(highlight) is dict:
            output.extend(highlight['items'])
        
        elif type(highlight) is str:
            output.append(highlight)
    return output

In [103]:
def extract(json_response: dict) -> pd.DataFrame:
    
    df = pd.DataFrame.from_records(json_response["jobs_results"])
    df['location'] = df['location'].str.replace('\([^()]*\)', "", regex=True)        # Remove unnecessary bracket in Location
    df['via'] = df['via'].str.replace('via', '', regex=False)                        # Remove via keyword from vi
    df = df.replace(to_replace=r'\n\n|\n•|\n', value=' ', regex=True)                # Remove new line and other unnecessary characters
    df.drop(columns=['thumbnail', 'extensions', 'related_links'], inplace=True)      # Remove unncecessary columns from the dataframe
    df.job_highlights = df.job_highlights.apply(merge_highlights)                    # Place dict/list of strings into a single list
    
    return df

In [104]:
def write_data(df_to_write: pd.DataFrame) -> None:
    file_name = 'data.csv'
    data_exists = os.path.isfile(file_name)
    if data_exists:
        df_to_write.to_csv(file_name, mode='a', index=False, header=False)
    else:
        df_to_write.to_csv(file_name, mode='w', index=False, header=True)

In [105]:
df = extract(results)
write_data(df)